In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#laster inn data og kvitter oss med noe 
data = pd.read_csv("..//Data/compas-scores-two-years.csv")
data = data[data["days_b_screening_arrest"] <= 30]
data = data[data["days_b_screening_arrest"] >= -30]

In [3]:
df = pd.DataFrame({
    "kvinne": data.sex == "Female",
    "hvit": data.race == "Caucasian",
    "svart": data.race == "African-American",
    "tilbakefall": data.two_year_recid == 1,
    "predikert_tilbakefall": data.score_text != "Low",
    "lav_score": data.score_text == "Low",
    "medium_score": data.score_text == "Medium",
    "høy_score": data.score_text == "High",})

Vi ønsker nå å se på data for følgende grupper:
* Hele befolkningen
* Kvinner 
* Menn 
* Hvite
* Svarte

Vi undersøker raten av falske positive, sanne positive, falske negative og sanne negative. I tillegg ønsker vi å se på sannsynligheten for at gruppen residiverer (tilbakefall) gitt at de er blitt predikert til å gjøre det. 

In [10]:
def falsk_positiv(gruppe):
    """Regner ut raten av falske positive for en gitt gruppe
    -----
    Arguments:
        gruppe: array med True/False verdier for dataframe
    
    Return:
        float: raten av falske positive i prosent med én desimal. 
    """
    # FP / (FP + SN)
    return float(f'{len(df[gruppe & df.predikert_tilbakefall & ~df.tilbakefall]) / len(df[gruppe & ~df.tilbakefall]) * 100:.1f}')

def sann_positiv(gruppe):
    """Regner ut raten av sanne positive for en gitt gruppe
    -----
    Arguments:
        gruppe: array med True/False verdier for dataframe
    
    Return:
        float: raten av sanne positive i prosent med én desimal.
    """
    # SP / (SP + FN)
    # sensitivitet
    return float(f'{len(df[gruppe & df.predikert_tilbakefall & df.tilbakefall]) / len(df[gruppe & df.tilbakefall]) * 100:.1f}')

def falsk_negativ(gruppe):
    """Regner ut raten av falske negative for en gitt gruppe
    -----
    Arguments:
        gruppe: array med True/False verdier for dataframe
    
    Return:
        float: raten av falske negative i prosent med én desimal.
    """
    # FN / (FN + SP)
    return float(f'{len(df[gruppe & ~df.predikert_tilbakefall & df.tilbakefall]) / len(df[gruppe & df.tilbakefall]) * 100:.1f}')

def sann_negativ(gruppe):
    """Regner ut raten av sanne negative for en gitt gruppe
    -----
    Arguments:
        gruppe: array med True/False verdier for dataframe
    
    Return:
        float: raten av sanne negative i prosent med én desimal.
    """
    # SN / (SN + FP)
    # spesifisitet
    return float(f'{len(df[gruppe & ~df.predikert_tilbakefall & ~df.tilbakefall]) / len(df[gruppe & ~df.tilbakefall]) * 100:.1f}')

def p_residiv_g_høy(gruppe):
    """Regner ut sannsynligheten for at en gruppe residiverer gitt at de har blitt predikert til å gjøre det
    -----
    Arguments:
        gruppe: array med True/False verdier for dataframe
    
    Return:
        float: sannsynlighet i prosent med én desimal.
    """
    return float(f'{len(df[gruppe & df.predikert_tilbakefall & df.tilbakefall]) / len(df[gruppe & df.predikert_tilbakefall]) * 100:.1f}')

In [9]:
alle = df.index > -1

In [8]:
functions = [falsk_positiv, falsk_negativ, sann_positiv, sann_negativ, p_residiv_g_høy]
grouplist = [alle, df.kvinne, ~df.kvinne, df.hvit, df.svart]
namelist = ['Alle', 'Kvinner', 'Menn', 'Hvite', 'Svarte']
d = {}
for name, group in zip(namelist, grouplist):
    inner_dict = {}
    for function in functions:
        inner_dict[function.__name__] = function(group)
    d[name] = inner_dict
d

{'Alle': {'falsk_positiv': 30.3,
  'falsk_negativ': 38.3,
  'sann_positiv': 61.7,
  'sann_negativ': 69.7,
  'p_residiv_g_høy': 63.0},
 'Kvinner': {'falsk_positiv': 30.2,
  'falsk_negativ': 40.4,
  'sann_positiv': 59.6,
  'sann_negativ': 69.8,
  'p_residiv_g_høy': 51.7},
 'Menn': {'falsk_positiv': 30.3,
  'falsk_negativ': 37.9,
  'sann_positiv': 62.1,
  'sann_negativ': 69.7,
  'p_residiv_g_høy': 65.4},
 'Hvite': {'falsk_positiv': 22.0,
  'falsk_negativ': 49.6,
  'sann_positiv': 50.4,
  'sann_negativ': 78.0,
  'p_residiv_g_høy': 59.5},
 'Svarte': {'falsk_positiv': 42.3,
  'falsk_negativ': 28.5,
  'sann_positiv': 71.5,
  'sann_negativ': 57.7,
  'p_residiv_g_høy': 65.0}}

Spesifisiteten (sanne negative) sier oss noe om hvor bra COMPAS klarer å identifisere personer som ikke kommer til å få tilbakefall. Sensitiviteten (sanne positive) sier noe om hvor bra COMPAS klarer å identifisere personer som kommer til å residivere. Det er først når begge disse tallene er høye det kan konkluderes med at algoritmen er god. Dersom det ene tallet er relativt høyt, imens den andre er relativt lavt sier oss bare om algoritmen er strengere i den ene eller andre retningen. For eksempel har vi for hvite at COMPAS har høy sann negativ rate og lav sann positiv rate. Ut ifra dette kan vi trekke frem en hypotese om at den generelt gir lav score for hvite.

Falske positive er personer som ikke residiverer, men som COMPAS predikerer vil gjøre det. Dette vil være mennesker som får strengere straff enn fortjent. Raten av falske positive vil være viktig å holde lav for individualistiske samfunn.   
Falske negative er personer vi kanskje slipper ut i samfunnet igjen, men som likevel begår en kriminell handling. Disse menneskene vil utgjøre en fare for samfunnet. Raten av falske negative vil være viktig å holde lav for kollektivistiske samfunn. 

Den betingede sannsynligheten sier noe om hvor sannsynlig det er at gruppen residiverer gitt at de 